In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.metrics import f1_score
from skopt.space import Real, Integer, Categorical
from skopt import gp_minimize

import utils.dev_config as dev_conf
import utils.preprocessing as prep
import utils.optimization as opt

In [2]:
dirs = dev_conf.get_dev_directories("../dev_paths.txt")
unified_dsets = ["unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data"]
matrisome_list = f"{dirs.data_dir}/matrisome/matrisome_hs_masterlist.tsv"

In [3]:
dset_idx = 0

In [4]:
matrisome_df = prep.load_matrisome_df(matrisome_list)

In [5]:
seed = 123
rand = np.random.RandomState()

# Load and filter survival data

In [6]:
event_code = {"Alive": 0, "Dead": 1}
covariate_cols = ["age_at_diagnosis", "race", "ethnicity"]
dep_cols = ["figo_stage"]
cat_cols = ["race", "ethnicity"]
survival_df = prep.load_survival_df(f"{dirs.data_dir}/{unified_dsets[dset_idx]}/survival_data.tsv", event_code)

In [7]:
filtered_survival_df = (
    prep.decode_figo_stage(survival_df[["sample_name"] + dep_cols + covariate_cols].dropna(), to="n")
        .pipe(pd.get_dummies, columns=cat_cols)
        .reset_index(drop = True)
        .pipe(prep.cols_to_front, ["sample_name", "figo_num"])
)
filtered_survival_df.columns = filtered_survival_df.columns.str.replace(' ', '_')
print(filtered_survival_df.shape)
# filtered_survival_df.head()

(255, 12)


# Load normalized matrisome count data

In [8]:
norm_matrisome_counts_df = pd.read_csv(f"{dirs.data_dir}/{unified_dsets[dset_idx]}/norm_matrisome_counts.tsv", sep='\t')
norm_filtered_matrisome_counts_t_df = prep.transpose_df(
    norm_matrisome_counts_df[["geneID"] + list(filtered_survival_df.sample_name)], "geneID", "sample_name"
)
print(norm_filtered_matrisome_counts_t_df.shape)
# norm_filtered_matrisome_counts_t_df.head()

(255, 1009)


# Join survival and count data

In [9]:
joined_df = (
    pd.merge(filtered_survival_df, norm_filtered_matrisome_counts_t_df, on="sample_name")
        .set_index("sample_name")
)
print(joined_df.shape)
# joined_df.head()

(255, 1019)


# Optimize model

In [10]:
rand.seed(seed)
x_df, y_df = prep.shuffle_data(joined_df, rand)

## Get baselines

In [11]:
def est_mc_baseline(y, labels, weights, metric, n=11):
    scores = []
    for _ in range(n):
        mc_yhat = np.random.choice(labels, size=y.shape[0], p = weights)
        scores.append(metric(y, mc_yhat))
    return np.array(scores)

rand.seed(seed)

label_value_counts_df = (
    pd.DataFrame(y_df.figo_num.value_counts()).reset_index()
        .rename(columns={"index": "label", "figo_num": "n"})
        .sort_values("n", ascending=False)
)

most_frequent_label = label_value_counts_df.label[0]
most_frequent_baseline = f1_score(y_df.values.squeeze(), np.repeat(most_frequent_label, y_df.shape[0]), average="weighted")

mc_baseline = est_mc_baseline(
    y=y_df.values.squeeze(),
    labels=label_value_counts_df.label.values,
    weights=label_value_counts_df.n.values / label_value_counts_df.n.values.sum(),
    metric=lambda y, yhat: f1_score(y, yhat, average="weighted"),
    n=1001
)

print(f"Most frequent baseline: {most_frequent_baseline}")
print(f"Monte Carlo baseline: {mc_baseline.mean()}")

Most frequent baseline: 0.3665158371040725
Monte Carlo baseline: 0.3661348131221232


## SMBO

In [12]:
def objective(h_params, X, y, loss_default, scoring_default, r, verbose=True):
    if verbose:
        print(h_params)
    model = GradientBoostingClassifier(
        loss=loss_default,
        learning_rate=h_params[0],
        n_estimators=h_params[1],
        max_depth=h_params[2],
        max_features=h_params[3],
        min_samples_split=h_params[4],
        min_samples_leaf=h_params[5],
        random_state=r
    )
    return -np.mean(cross_val_score(
        model,
        X,
        y,
        cv=KFold(n_splits=5),
        n_jobs=-1,
        scoring=scoring_default
    ))

In [13]:
space = [
    Real(1e-3, 1e-1, name="learning_rate"),
    Integer(int(1e2), int(1e3), name="n_estimators"),
    Integer(2, 5, name="max_depth"),
    Categorical(["auto", "sqrt", "log2"], name="max_features"),
    Integer(int(2), int(6), name="min_samples_split"),
    Integer(int(1), int(3), name="min_samples_leaf")
]
n_initial = 10 * len(space)
n_calls = 50 * len(space)

In [14]:
loss_default = "deviance"
scoring_default = "f1_weighted"
callback_file = f"{unified_dsets[dset_idx]}_opt_gbc_h_params_{scoring_default}.tsv"

try:
    os.remove(callback_file)
except OSError:
    pass

res = gp_minimize(
    lambda h_ps: objective(h_ps, x_df, y_df, loss_default, scoring_default, rand),
    space,
    verbose=True,
    random_state=rand,
    n_initial_points=n_initial,
    n_calls=n_calls,
    n_jobs=-1,
    callback=lambda x: opt.save_callback(x, callback_file, n = 5, sep="\t")
)

Iteration No: 1 started. Evaluating function at random point.
[0.0715825767317995, 486, 4, 'log2', 4, 3]
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 2.8113
Function value obtained: -0.4209
Current minimum: -0.4209
Iteration No: 2 started. Evaluating function at random point.
[0.04168151289819289, 622, 2, 'log2', 5, 2]
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 2.3421
Function value obtained: -0.4596
Current minimum: -0.4596
Iteration No: 3 started. Evaluating function at random point.
[0.025231168418438074, 725, 4, 'log2', 4, 1]
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 3.6834
Function value obtained: -0.4102
Current minimum: -0.4596
Iteration No: 4 started. Evaluating function at random point.
[0.0715206879574875, 485, 3, 'log2', 5, 2]
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 2.2576
Function value obtained: -0.4456
Current minimum: -0.4596
Iteration No: 5 started. Evaluating function 

/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 75 ended. Search finished for the next optimal point.
Time taken: 7.7019
Function value obtained: -0.4997
Current minimum: -0.5038
Iteration No: 76 started. Searching for the next optimal point.
[0.09942636523085548, 179, 2, 'sqrt', 6, 3]
Iteration No: 76 ended. Search finished for the next optimal point.
Time taken: 2.4855
Function value obtained: -0.4564
Current minimum: -0.5038
Iteration No: 77 started. Searching for the next optimal point.
[0.09998111739187887, 860, 5, 'log2', 4, 1]
Iteration No: 77 ended. Search finished for the next optimal point.
Time taken: 7.7115
Function value obtained: -0.4335
Current minimum: -0.5038
Iteration No: 78 started. Searching for the next optimal point.
[0.022296317741992946, 1000, 2, 'log2', 2, 1]
Iteration No: 78 ended. Search finished for the next optimal point.
Time taken: 5.5580
Function value obtained: -0.4654
Current minimum: -0.5038
Iteration No: 79 started. Searching for the next optimal point.
[0.1, 1000, 2, 'sqrt', 6, 3]


/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 79 ended. Search finished for the next optimal point.
Time taken: 8.3301
Function value obtained: -0.4788
Current minimum: -0.5038
Iteration No: 80 started. Searching for the next optimal point.
[0.1, 1000, 2, 'sqrt', 6, 3]


...

Iteration No: 298 ended. Search finished for the next optimal point.
Time taken: 12.3134
Function value obtained: -0.4921
Current minimum: -0.5134
Iteration No: 299 started. Searching for the next optimal point.
[0.1, 1000, 2, 'sqrt', 6, 3]


/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 299 ended. Search finished for the next optimal point.
Time taken: 12.1494
Function value obtained: -0.4915
Current minimum: -0.5134
Iteration No: 300 started. Searching for the next optimal point.
[0.1, 1000, 2, 'sqrt', 6, 3]


/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 300 ended. Search finished for the next optimal point.
Time taken: 12.0556
Function value obtained: -0.4984
Current minimum: -0.5134
